# Road Follower - Train Model(モデルの学習)

このnotebookは、入力画像を読み込み、ターゲットに対応するx、y値のセットを出力するようにニューラルネットワークを学習します。

road followerアプリのためにResNet18 neural networkアーキテクチャーモデルを学習するために、PyTorch deep learningフレームワークを使用します。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Create Dataset Instance(データセットインスタンスの作成)

``__len__`` と ``__getitem__``関数が実装されたカスタマイズされた``torch.utils.data.Dataset`` の実装を生成します。このクラスは、画像をロードするための役割と、画像ファイル名からx、y値の値をパースして取得します。`torch.utils.data.Dataset``を実装する事で、すべてのtorchデータユーティリティを使用する事ができます。

いくつかの変換（カラージッターなど）をデータセットにハードコーディングしました。ランダムな水平反転をオプションにしました（「右にとどまる」必要がある道路のように、非対称の経路をたどる場合）。ロボットが何らかの規則に従っているかどうかが重要でない場合は、フリップを有効にしてデータセットを拡張できます

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### Split dataset into train and test sets(トレーニングとテスト用のデータセットに分ける)
データセットをロードしたら、トレーニング用とテスト用のデータセットに分割します。この例では、トレーニング用が90%, テスト用が10%で分けます。テストセットは、トレーニングするモデルの精度を検証するために使用されます。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches(バッチでデータを読み込むためのdata loadersを作成)

`` DataLoader``クラスは、並行したサブプロセスを使用して、データのシャッフル、バッチでのデータロードのために使用します。この例では、64のバッチサイズを使用します。バッチサイズは、GPUで使用可能なメモリに基づいており、モデルの精度に影響を与える可能性があります。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define Neural Network Model (Neural netwrk modelの定義)

PyToch TouchVisionで使用可能なResNet-18モデルを使用します。

*転移学習*と呼ばれる手法で、利用可能なデータがはるかに少ない状態で新しいタスクをおこなうために、事前にトレーニングされたモデル（数百万の画像でトレーニングされた）を再利用する事ができます。

ResNet-18の詳細: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

転移学習の詳細: https://www.youtube.com/watch?v=yofjFQddwHE

ResNetのモデルは、`in_features``として、512を最終レイヤーに全結合します。そして、``out_features``2(x,y座標)にして回帰のトレーニングをおこないます。

最後に、GPUで実行するために、モデルを転送します。

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

70エポック学習し、損失が減少した場合は、ベストなモデルを保存します。

In [ ]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())
epoch_list = []
train_loss_list = []
test_loss_list = []

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%d : train loss %f, test_loss %f' % (epoch, train_loss, test_loss))
    epoch_list.append(epoch) 
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

学習が完了すると、Live demo notebookで推論に使う``best_steering_model_xy.pth``が生成されます。

## Next(次)

次は、live_demo.ipynbで学習済みモデルで自動走行します。

TensorRTを試したい人は、trtフォルダの中のconvert_trt.ipynbを実行し、学習済みモデルをTensorRT形式に変換し、live_demo.ipynbを実行し自動走行します。